In [2]:
import mlflow

from data_ingest import get_data

c:\Users\Ripple\Desktop\mlops\implement_dvc_emotion_detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
        

train_df, test_df = get_data("parulpandey/emotion-dataset")


In [ ]:
# sadness (0), joy (1), love (2), anger (3), fear (4).

In [4]:
train_df['label'].value_counts()

label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64

In [5]:
train_df=train_df[train_df["label"].isin([1,0])]
test_df = test_df[test_df['label'].isin([1,0])]

In [6]:
from preprocess import remove_short_sentences,normalized_sentence

c:\Users\Ripple\Desktop\mlops\implement_dvc_emotion_detection\notebooks\experiments\preprocess.py:50: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub("\s+", " ", text)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ripple\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ripple\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:


train_df["text"] = train_df["text"].apply(normalized_sentence)
test_df["text"] = test_df["text"].apply(normalized_sentence)


In [8]:
train_data = remove_short_sentences(train_df, "text", 4)
test_data = remove_short_sentences(test_df, "text", 4)


In [9]:
X_train = train_df["text"]
X_test = test_df["text"]


y_train = train_df["label"]
y_test = test_df["label"]


In [10]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer,accuracy_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [11]:
pipeline = Pipeline([
    ("feature_transformer",None),
    ("model",LogisticRegression(solver="liblinear"))
])

In [12]:

param_grid = [
    {
        "feature_transformer": [CountVectorizer()],
        "feature_transformer__max_features": [100, 500, 1000],
        "model__penalty":["l1","l2"]
    },
    {
        "feature_transformer": [TfidfVectorizer()],
        "feature_transformer__max_features": [100, 500, 1000],
        "model__penalty":["l1","l2"]
    },

]

In [13]:
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score, average="weighted"),
    "recall": make_scorer(recall_score, average="weighted"),
}


In [14]:
gs = GridSearchCV(
    estimator=pipeline, param_grid=param_grid, scoring=scoring, refit="accuracy"
)

In [15]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.sklearn.autolog(max_tuning_runs=None)
mlflow.set_experiment(experiment_name="baseline_exp")

with mlflow.start_run():
    gs.fit(X_train,y_train,)

KeyboardInterrupt: 